Purpose: Time-series / sequence model using LSTM for behavior-based fraud detection.

In [1]:
# %%
# 03-lstm.ipynb — LSTM model for sequence-based fraud detection
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, roc_auc_score

# Load processed data (you can reuse or reshape for LSTM)
data = np.load('../data/processed_data.npz')
X_res = data['X_res']
y_res = data['y_res']

# Reshape to (samples, timesteps, features)
X_res = X_res.reshape(X_res.shape[0], 1, X_res.shape[1])

# Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

# Model
model = models.Sequential([
    layers.LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])
model.summary()

# Train
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=256)
model.save('../models/lstm_model.h5')

# Evaluate
y_pred = (model.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_pred))


e:\Fourise\Fraud-Detaction\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        17,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,521 (76.25 KB)

 Trainable params: 19,521 (76.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
1777/1777 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.6821 - auc: 0.7372 - loss: 0.6037 - val_accuracy: 0.6909 - val_auc: 0.7512 - val_loss: 0.5944
Epoch 2/5
1777/1777 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.6883 - auc: 0.7522 - loss: 0.5931 - val_accuracy: 0.6892 - val_auc: 0.7610 - val_loss: 0.5857
Epoch 3/5
1777/1777 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.6960 - auc: 0.7593 - loss: 0.5853 - val_accuracy: 0.7003 - val_auc: 0.7704 - val_loss: 0.5757
Epoch 4/5
1777/1777 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.7113 - auc: 0.7699 - loss: 0.5731 - val_accuracy: 0.7304 - val_auc: 0.7839 - val_loss: 0.5595
Epoch 5/5
1777/1777 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7243 - auc: 0.7824 - loss: 0.5597 - val_accuracy: 0.7321 - val_auc: 0.7947 - val_loss: 0.5456


3554/3554 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step
              precision    recall  f1-score   support

           0       0.73      0.73      0.73     56750
           1       0.73      0.74      0.73     56976

    accuracy                           0.73    113726
   macro avg       0.73      0.73      0.73    113726
weighted avg       0.73      0.73      0.73    113726

ROC-AUC: 0.7321186520145432
